## Importando Bibliotecas

In [19]:
import tensorflow as tf 
from tensorflow import keras
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Input, Dense
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from keras.optimizers import Adam
from keras.models import Model
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

## Pre-processamento dos dados
**Carregando os dados de treino e teste:**

In [20]:
train_images = pd.read_csv('../fashion-mnist_train.csv')
test_images = pd.read_csv('../fashion-mnist_test.csv')

**Separando as labels do conjuntos:**

In [21]:
train_labels=train_images.loc[:, ['label']]
train_images=train_images.drop(['label'], axis=1)
test_labels=test_images.loc[:, ['label']]
test_images=test_images.drop(['label'], axis=1)

**Normalizando os data-sets:**

In [22]:
train_images = train_images / 255.0
test_images = test_images / 255.0


----------------------
## Construção do modelo base
**Esta rede será utilizada posteriomente para comparações entre os modelos de dimensionalidade reduzida utilizando PCA e autoencoder.**

**Definição da estrutura do modelo por meio do Keras Sequential:**

In [23]:
model = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(784)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [25]:
model.fit(train_images, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 5s 107us/sample - loss: 0.5207 - accuracy: 0.8132 - val_loss: 0.4140 - val_accuracy: 0.8540
Epoch 2/5
48000/48000 [==============================] - 5s 96us/sample - loss: 0.3872 - accuracy: 0.8602 - val_loss: 0.4046 - val_accuracy: 0.8549
Epoch 3/5
48000/48000 [==============================] - 5s 95us/sample - loss: 0.3549 - accuracy: 0.8705 - val_loss: 0.3584 - val_accuracy: 0.8703
Epoch 4/5
48000/48000 [==============================] - 5s 95us/sample - loss: 0.3289 - accuracy: 0.8785 - val_loss: 0.3521 - val_accuracy: 0.8712
Epoch 5/5
48000/48000 [==============================] - 5s 98us/sample - loss: 0.3119 - accuracy: 0.8836 - val_loss: 0.3551 - val_accuracy: 0.8720


**Verificação dos resultados do modelo:**

In [26]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 35us/sample - loss: 0.3432 - accuracy: 0.8751
Model - 3 layers - test loss: 34.32043188452721
Model - 3 layers - test accuracy: 87.51000165939331


-----------------------------
## Redução de Dimensionalidade Usando Principal component analysis (PCA)

**Preprocessamento de dados (Feature Scaling)**

In [27]:
scaler = StandardScaler()
scaler.fit(train_images)

train_images_r = scaler.transform(train_images)
test_images_r = scaler.transform(test_images)

**Definição do número de componentes:**

In [28]:
pca = PCA(n_components=3)

**Construção dos novos datasets de treino e de teste com dimensões reduzidas usando o PCA:**

In [29]:
pca.fit(train_images_r)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [30]:
train_images_r = pca.transform(train_images_r)
test_images_r = pca.transform(test_images_r)

**Preprocessamento: normalização e alteração do tipo dos dados de numpy array para pandas dataframe:**

In [41]:
#esse conjunto rr sera utilizado pela rede neural
train_images_rr = normalize(train_images_r)
test_images_rr = normalize(test_images_r)
train_images_rr = pd.DataFrame(data=train_images_rr)
test_images_rr = pd.DataFrame(data=test_images_rr)

#esse conjunto rr sera utilizado posteriormente para a clusterizacao 
train_images_r = pd.DataFrame(data=train_images_r)
test_images_r= pd.DataFrame(data=test_images_r)

**Definição da estrutura do modelo por meio do Keras Sequential que receberá os dados com dimensões reduzidas pelo PCA:**

In [42]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 64)                256       
_________________________________________________________________
dense_16 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1290      
Total params: 9,866
Trainable params: 9,866
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [43]:
model_r.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Normalização dos dados e Treino do modelo:**

In [44]:
model_r.fit(train_images_rr, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 56us/sample - loss: 1.0970 - accuracy: 0.5763 - val_loss: 1.0383 - val_accuracy: 0.5920
Epoch 2/5
48000/48000 [==============================] - 3s 58us/sample - loss: 1.0163 - accuracy: 0.5989 - val_loss: 1.0267 - val_accuracy: 0.5884
Epoch 3/5
48000/48000 [==============================] - 3s 55us/sample - loss: 1.0059 - accuracy: 0.6035 - val_loss: 1.0088 - val_accuracy: 0.6029
Epoch 4/5
48000/48000 [==============================] - 3s 61us/sample - loss: 1.0017 - accuracy: 0.6018 - val_loss: 1.0114 - val_accuracy: 0.5985
Epoch 5/5
48000/48000 [==============================] - 3s 60us/sample - loss: 0.9987 - accuracy: 0.6048 - val_loss: 1.0157 - val_accuracy: 0.5982


**Verificação dos resultados do modelo:**

In [45]:
test_loss, test_acc = model_r.evaluate(test_images_rr, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 29us/sample - loss: 1.0068 - accuracy: 0.6036
Model - 3 layers - test loss: 100.67673736572264
Model - 3 layers - test accuracy: 60.360002517700195


------------------------------------------------------

## Redução de Dimensionalidade Usando Autoencoders

In [46]:
# this is the size of our encoded representations
encoding_dim = 2
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
encoder = Model(input_img, encoded)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [47]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [48]:
#REMOVER troque o 'adadelta' pelo 'adam'
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [49]:
autoencoder.fit(train_images, train_images,epochs=5,shuffle=True,validation_data=(test_images, test_images))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 9s 145us/step - loss: 0.4407 - val_loss: 0.4098
Epoch 2/5
60000/60000 [==============================] - 8s 126us/step - loss: 0.4008 - val_loss: 0.3924
Epoch 3/5
60000/60000 [==============================] - 8s 127us/step - loss: 0.3880 - val_loss: 0.3824
Epoch 4/5
60000/60000 [==============================] - 7s 120us/step - loss: 0.3806 - val_loss: 0.3775
Epoch 5/5
60000/60000 [==============================] - 8s 130us/step - loss: 0.3778 - val_loss: 0.3760


In [50]:
encoded_imgs_train = encoder.predict(train_images)
encoded_imgs_test = encoder.predict(test_images)


In [51]:
train_images_a = pd.DataFrame(data=encoded_imgs_train)
test_images_a =pd.DataFrame(data=encoded_imgs_test)

In [52]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(2)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 64)                192       
_________________________________________________________________
dense_19 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_20 (Dense)             (None, 10)                1290      
Total params: 9,802
Trainable params: 9,802
Non-trainable params: 0
_________________________________________________________________


In [53]:
model_r.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
model_r.fit(train_images_a, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 72us/sample - loss: 1.2688 - accuracy: 0.4771 - val_loss: 1.1704 - val_accuracy: 0.5268
Epoch 2/5
48000/48000 [==============================] - 3s 68us/sample - loss: 1.1502 - accuracy: 0.5331 - val_loss: 1.1397 - val_accuracy: 0.5432
Epoch 3/5
48000/48000 [==============================] - 3s 67us/sample - loss: 1.1348 - accuracy: 0.5367 - val_loss: 1.1407 - val_accuracy: 0.5415
Epoch 4/5
48000/48000 [==============================] - 3s 68us/sample - loss: 1.1294 - accuracy: 0.5397 - val_loss: 1.1271 - val_accuracy: 0.5429
Epoch 5/5
48000/48000 [==============================] - 4s 73us/sample - loss: 1.1248 - accuracy: 0.5431 - val_loss: 1.1481 - val_accuracy: 0.5393


In [55]:
test_loss, test_acc = model_r.evaluate(test_images_a, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 28us/sample - loss: 1.1461 - accuracy: 0.5363
Model - 3 layers - test loss: 114.61299938201905
Model - 3 layers - test accuracy: 53.6300003528595


------------------------------------------------------

## Clustering com K-means

Optamos pela utilização das imagens reduzidas por meio do PCA, já que obtiveram por meio deste modelo os melhores resultados de predição na rede neural.

In [56]:
train_images_k = train_images_r
test_images_k = test_images_r

Construção do modelo do k-means com 10 clusters.

In [65]:
kmean_model = KMeans(n_clusters=10, random_state=10)
y_km = kmean_model.fit(train_images_k)
labels = kmean_model.labels_

In [66]:
cluster_centroids = kmean_model.cluster_centers_

In [67]:
silhouette_score(X = train_images_k, labels = labels, random_state = 10)

0.36336111205020194

In [68]:
metrics.v_measure_score(train_labels.values.ravel(), labels, beta=1.0)

0.4455785750478227

---------------------------------
## Clustering com o DBScan
